#### Code to validate models

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [42]:
import os
import cv2
import json
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import *
from collections import Counter
from tqdm.notebook import tqdm

In [43]:
from params import *

In [44]:
from data.preparation import prepare_dataframe, handle_duplicates
from data.dataset import CovidDetDataset, CovidClsDataset
from data.transforms import get_transfos_det, get_transfos_cls

from model_zoo.models import get_model

from utils.plot import plot_sample

from utils.logger import Config

from utils.metrics import per_class_average_precision_score, study_level_map

## Data

### Load

In [45]:
EXP_FOLDERS = [
#     LOG_PATH + "2021-07-25/10/",
#     LOG_PATH + "2021-07-26/1/",
#     LOG_PATH + "2021-07-26/2/",
    LOG_PATH + "2021-07-27/15/",
#     LOG_PATH + "2021-07-28/0/",
]

EXP_FOLDER = EXP_FOLDERS[-1]

USE_TTA = True

In [46]:
config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))

In [47]:
if USE_TTA:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img_flip.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study_flip.npy") for f in EXP_FOLDERS], 0)
else:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study.npy") for f in EXP_FOLDERS], 0)

In [48]:
df = pd.read_csv(EXP_FOLDER + "data.csv")

pred_cols = [c + "_pred" for c in CLASSES]
df[pred_cols] = pred_oof_study
df['pred_img'] = pred_oof_img

In [49]:
auc = roc_auc_score(df['img_target'], pred_oof_img)
print(f'Image AUC : {auc :.3f}')
acc = accuracy_score(df['img_target'], pred_oof_img > 0.5)
print(f'Image Acc : {acc :.3f}')

Image AUC : 0.905
Image Acc : 0.845


In [50]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.4f}')

Study mAP : 0.3741


In [51]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values, agg=np.min)
print(f'Study mAP : {study_map :.4f}')

Study mAP : 0.3742


### Post-process

#### Img merging

In [13]:
groups = df[['study_id', 'pred_img']].groupby('study_id').mean().rename(
    columns={'pred_img': 'pred_img_merged'}
).reset_index()
df_ = df.merge(groups, on="study_id", how="left")

In [14]:
auc = roc_auc_score(df_['img_target'], df_['pred_img_merged'])
print(f'Image AUC : {auc :.3f}')
acc = accuracy_score(df_['img_target'], df_['pred_img_merged'] > 0.5)
print(f'Image Acc : {acc :.3f}')

Image AUC : 0.905
Image Acc : 0.845


#### Study using img

In [15]:
df_ = df.copy()

df_['negative_pred'] *= 1 - df_['pred_img'] 
df_['typical_pred'] *= df_['pred_img'] 
# df_['indeterminate_pred'] *= df_['pred_img'] 
# df_['atypical_pred'] *= df_['pred_img'] 


study_map = study_level_map(df_[pred_cols].values, df_[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.4f}')

Study mAP : 0.3745


In [16]:
df_study = df[['study_id'] + pred_cols + CLASSES + ['pred_img']].groupby('study_id').agg(np.mean)

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study.loc[df_study['pred_img'] > 0.9, 'negative_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.1, 'negative_pred'] *= 2
# df_study.loc[df_study['pred_img'] > 0.9, 'typical_pred'] *= 2
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 2
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 2

per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)

array([0.79027104, 0.84551651, 0.28282801, 0.3261376 ])

In [17]:
df_study = df[
    ['study_id'] + pred_cols + CLASSES + ['img_target', 'pred_img']
].groupby('study_id').agg(np.mean).copy()

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study['typical_pred'] *= df_study['pred_img'] 
# df_study['indeterminate_pred'] *= df_study['pred_img'] 
# df_study['atypical_pred'] *= df_study['pred_img'] 

df_study.loc[df_study['pred_img'] > 0.75, 'negative_pred'] *= 0.5
df_study.loc[df_study['pred_img'] < 0.2, 'negative_pred'] *= 2

df_study.loc[df_study['pred_img'] > 0.75, 'typical_pred'] *= 1.2
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 1.1
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 1.1

df_study.loc[df_study['pred_img'] < 0.25, 'typical_pred'] *= 0.8
# df_study.loc[df_study['pred_img'] < 0.2, 'indeterminate_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.2, 'atypical_pred'] *= 0.9

accs = per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)
np.round(accs, 4), np.round(np.mean(accs) * 2/3, 4)

(array([0.7909, 0.8459, 0.2828, 0.3261]), 0.3743)

## Results

In [54]:
pred_oof_study = pd.read_csv('../output/OOF_study_only_EBV2M_768.csv')
# pred_oof_study['id'] = pred_oof_study['id'].apply(lambda x: x.split('_')[0])

In [23]:
pred_oof_study = pd.read_csv('../output/OOF_study_only_EBV2M_768.csv')
pred_oof_study['id'] = pred_oof_study['id'].apply(lambda x: x.split('_')[0])

df_study = df[['study_id'] + pred_cols + CLASSES].groupby('study_id').agg(np.mean)
df_study = df_study.merge(pred_oof_study, how="left", left_on="study_id", right_on="id").dropna()

In [24]:
len(df_study), len(df)

(6032, 6333)

In [25]:
def proc(x):
    x = x.split('0 0 1 1')[:4]
    x = [float(y.strip().split(' ')[1]) for y in x]
    return x

pred_oof_study = np.array(df_study['PredictionString'].apply(proc).values.tolist())
# pred_oof_study = np.random.random(pred_oof_study.shape)
df_study[pred_cols] = pred_oof_study

In [26]:
df_g = df[['study_id'] + pred_cols].groupby('study_id').mean().reset_index()
df_study = df_study.merge(df_g, how="left", left_on="id", right_on="study_id", suffixes=['', '_theo'])

In [27]:
per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES) * 2 / 3

0.3781226084618286

In [28]:
per_class_average_precision_score(
    df_study[[p + "_theo" for p in pred_cols]].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES) * 2 / 3

0.3741632323479383